# Read me

__Brand N-size Calculator__
<br> _Created by: Cameron Miller_
<br> _Last updated: 05.30.24_

This tool can be used to estimate the number of exposures per brand when the following parameters are known:
- Total expected sample size
- Number of brands in study
- Estimated awareness level for each brand
- Number of brands assigned per respondent
- Assignment method (least fill or randomization)
- Brand prioritization (if any)

In the 'User inputs' section, these parameters are entered as prompted and the script outputs the following in the 'Run' section, taking into consideration the user's settings:
- The estimated number of exposures for each brand
- The number of respondents assigned 0 brands, 1 brand, 2 brands, etc. up to the number of possible brands assigned

No revisions to actual code are needed in the 'User inputs' section, and nothing should be inputted/changed at all in the 'Setup' or 'Run' sections.

# User inputs

In [ ]:
# Total sample size
while True:
    try:
        N = int(input("Enter your expected total sample size (e.g., 1000): "))
        break
    except ValueError:
        print("Please make sure you input a numeric value with no commas or extra characters.")

In [ ]:
# Total number of brands
while True:
    try:
        num_brands = int(input("Enter the number of brands in your study (e.g., 10): "))
        break
    except ValueError:
        print("Please make sure you input a numeric value with no commas or extra characters.")

In [ ]:
# Awareness levels of each brand
brand_awareness_levels = []

for i in range(num_brands):
    while True:
        try:
            brand_awareness_level = float(input(f"Enter the estimated awareness level of Brand {i+1} as a decimal between 0 and 1 (e.g., 0.25 for 25%): "))
            if 0 <= brand_awareness_level <= 1:
                brand_awareness_levels.append(brand_awareness_level)
                break
            else:
                print("Please enter a numeric value as a decimal between 0 and 1.")
        except ValueError:
            print("Please enter a numeric value as a decimal between 0 and 1.")

In [ ]:
# Number of brands assigned per respondent
while True:
    try:
        num_brands_assigned = int(input("Enter the number of brands assigned per respondent (e.g., 2): "))
        if num_brands_assigned <= num_brands:
            break
        else:
            print(f"The number of brands assigned per respondent must be less than or equal to the total number of brands ({num_brands}).")
    except ValueError:
        print("Please make sure you input a numeric value with no commas or extra characters.")

In [ ]:
# Method of assignment
class InvalidAssignmentMethodError(ValueError):
    pass

while True:
    try:
        assignment_method = input("Enter the method of brand assignment, either least fill or random: ").strip().replace(" ", "").lower()
        if assignment_method not in ["leastfill", "random"]:
            raise InvalidAssignmentMethodError(f"Invalid assignment method: {assignment_method}. Please enter least fill or random.")
        break
    except InvalidAssignmentMethodError as e:
        print(e)

In [ ]:
class InvalidBrandError(ValueError):
    pass

while True:
    try:
        prioritized_brands_input = input("Enter any prioritized brands separated by commas (e.g., Brand 1, Brand 3) or leave blank if none: ").strip()
        if not prioritized_brands_input:
            prioritized_brands = []
            break
        prioritized_brands = [brand.strip().lower().capitalize().replace(" ", "") for brand in prioritized_brands_input.split(',')]
        invalid_brands = [brand for brand in prioritized_brands if not (brand.startswith("Brand") and brand[5:].isdigit() and 1 <= int(brand[5:]) <= num_brands)]
        if invalid_brands:
            raise InvalidBrandError(f"Invalid brand(s) found ({', '.join(invalid_brands)}). Please enter brands in the format 'Brand #' where # is between 1 and {num_brands}.")
        break
    except InvalidBrandError as e:
        print(e)

# Setup

In [ ]:
# Import packages
import random

In [ ]:
# Create the brand awareness dictionary
brand_awareness = {f"Brand{i+1}": awareness for i, awareness in enumerate(brand_awareness_levels)}

In [ ]:
# Calculate initial expected assignments
def calculate_expected_assignments(N, brand_awareness):
    return {brand: int(N * awareness) for brand, awareness in brand_awareness.items()}

In [ ]:
# Initialize assignment counts
def initialize_assignments(brand_awareness):
    return {brand: 0 for brand in brand_awareness}

In [ ]:
# Assign a respondent to up to `num_brands_assigned` brands using least fill logic
def assign_brands_least_fill(assignments, expected_assignments, num_brands_assigned, prioritized_brands):
    assigned_brands = []
    # First assign to prioritized brands
    for brand in prioritized_brands:
        if len(assigned_brands) < num_brands_assigned and assignments[brand] < expected_assignments[brand]:
            assigned_brands.append(brand)
            assignments[brand] += 1
    # Assign remaining slots using least fill logic
    if len(assigned_brands) < num_brands_assigned:
        sorted_brands = sorted(assignments.items(), key=lambda x: x[1])
        for brand, _ in sorted_brands:
            if len(assigned_brands) < num_brands_assigned and brand not in assigned_brands and assignments[brand] < expected_assignments[brand]:
                assigned_brands.append(brand)
                assignments[brand] += 1
    return assigned_brands

In [ ]:
# Assign a respondent to up to `num_brands_assigned` brands using pure randomization
def assign_brands_random(assignments, expected_assignments, num_brands_assigned, brand_awareness, prioritized_brands):
    assigned_brands = []
    # First assign to prioritized brands
    for brand in prioritized_brands:
        if len(assigned_brands) < num_brands_assigned and assignments[brand] < expected_assignments[brand]:
            assigned_brands.append(brand)
            assignments[brand] += 1
    # Assign remaining slots randomly
    if len(assigned_brands) < num_brands_assigned:
        available_brands = [brand for brand, count in expected_assignments.items() if assignments[brand] < count and brand not in assigned_brands]
        random_brands = random.sample(available_brands, min(num_brands_assigned - len(assigned_brands), len(available_brands)))
        for brand in random_brands:
            assigned_brands.append(brand)
            assignments[brand] += 1
    return assigned_brands

In [ ]:
# Distribute respondents
def distribute_respondents(N, brand_awareness, num_brands_assigned, assignment_method, prioritized_brands):
    expected_assignments = calculate_expected_assignments(N, brand_awareness)
    assignments = initialize_assignments(brand_awareness)
    respondents_assignment_count = [0] * N
    for i in range(N):
        if assignment_method == 'leastfill':
            assigned_brands = assign_brands_least_fill(assignments, expected_assignments, num_brands_assigned, prioritized_brands)
        elif assignment_method == 'random':
            assigned_brands = assign_brands_random(assignments, expected_assignments, num_brands_assigned, brand_awareness, prioritized_brands)
        respondents_assignment_count[i] = len(assigned_brands)
    assignment_distribution = [respondents_assignment_count.count(i) for i in range(num_brands_assigned + 1)]
    return assignments, assignment_distribution

# Run

In [ ]:
assignments, assignment_distribution = distribute_respondents(N, brand_awareness, num_brands_assigned, assignment_method, prioritized_brands)

html_output = "<h2>Brand Assignments</h2>"
html_output += "<table><tr><th>Brand</th><th>Count</th></tr>"
for brand, count in assignments.items():
    html_output += f"<tr><td>{brand}</td><td>{count}</td></tr>"
html_output += "</table>"

html_output += "<h2>Respondent Assignments</h2>"
html_output += "<table><tr><th>Brands Assigned</th><th>Count</th></tr>"
for i, count in enumerate(assignment_distribution):
    html_output += f"<tr><td>{i}</td><td>{count}</td></tr>"
html_output += "</table>"

from IPython.display import display, HTML
display(HTML(html_output))